In [1]:

#Initial import of libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from nltk import download, FreqDist, WordNetLemmatizer
from nltk.tokenize.regexp import wordpunct_tokenize
import string

#Download punkt and wordnet
download("wordnet")
download("punkt")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ckadam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ckadam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
wnl = WordNetLemmatizer()  
converter = lambda x: wnl.lemmatize(str.strip(x, string.punctuation + "0123456789").lower())

#Saved “Long Stopword List” from http://www.ranks.nl/stopwords into string
longStopwordList = "a<br>able<br>about<br>above<br>abst<br>accordance<br>according<br>accordingly<br>across<br>act<br>actually<br>added<br>adj<br>affected<br>affecting<br>affects<br>after<br>afterwards<br>again<br>against<br>ah<br>all<br>almost<br>alone<br>along<br>already<br>also<br>although<br>always<br>am<br>among<br>amongst<br>an<br>and<br>announce<br>another<br>any<br>anybody<br>anyhow<br>anymore<br>anyone<br>anything<br>anyway<br>anyways<br>anywhere<br>apparently<br>approximately<br>are<br>aren<br>don>won<br>arent<br>arise<br>around<br>as<br>aside<br>ask<br>asking<br>at<br>auth<br>available<br>away<br>awfully<br>b<br>back<br>be<br>became<br>because<br>become<br>becomes<br>becoming<br>been<br>before<br>beforehand<br>begin<br>beginning<br>beginnings<br>begins<br>behind<br>being<br>believe<br>below<br>beside<br>besides<br>between<br>beyond<br>biol<br>both<br>brief<br>briefly<br>but<br>by<br>c<br>ca<br>came<br>can<br>cannot<br>can't<br>cause<br>causes<br>certain<br>certainly<br>co<br>com<br>come<br>comes<br>contain<br>containing<br>contains<br>could<br>couldnt<br>d<br>date<br>did<br>didn't<br>different<br>do<br>does<br>doesn't<br>doing<br>done<br>don't<br>down<br>downwards<br>due<br>during<br>e<br>each<br>ed<br>edu<br>effect<br>eg<br>eight<br>eighty<br>either<br>else<br>elsewhere<br>end<br>ending<br>enough<br>especially<br>et<br>et-al<br>etc<br>even<br>ever<br>every<br>everybody<br>everyone<br>everything<br>everywhere<br>ex<br>except<br>f<br>far<br>few<br>ff<br>fifth<br>first<br>five<br>fix<br>followed<br>following<br>follows<br>for<br>former<br>formerly<br>forth<br>found<br>four<br>from<br>further<br>furthermore<br>g<br>gave<br>get<br>gets<br>getting<br>give<br>given<br>gives<br>giving<br>go<br>goes<br>gone<br>got<br>gotten<br>h<br>had<br>happens<br>hardly<br>has<br>hasn't<br>have<br>haven't<br>having<br>he<br>hed<br>hence<br>her<br>here<br>hereafter<br>hereby<br>herein<br>heres<br>hereupon<br>hers<br>herself<br>hes<br>hi<br>hid<br>him<br>himself<br>his<br>hither<br>home<br>how<br>howbeit<br>however<br>hundred<br>i<br>id<br>ie<br>if<br>i'll<br>im<br>immediate<br>immediately<br>importance<br>important<br>in<br>inc<br>indeed<br>index<br>information<br>instead<br>into<br>invention<br>inward<br>is<br>isn't<br>it<br>itd<br>it'll<br>its<br>itself<br>i've<br>j<br>just<br>k<br>keep<br>keeps<br>kept<br>kg<br>km<br>know<br>known<br>knows<br>l<br>largely<br>last<br>lately<br>later<br>latter<br>latterly<br>least<br>less<br>lest<br>let<br>lets<br>like<br>liked<br>likely<br>line<br>little<br>'ll<br>look<br>looking<br>looks<br>ltd<br>m<br>made<br>mainly<br>make<br>makes<br>many<br>may<br>maybe<br>me<br>mean<br>means<br>meantime<br>meanwhile<br>merely<br>mg<br>might<br>million<br>miss<br>ml<br>more<br>moreover<br>most<br>mostly<br>mr<br>mrs<br>much<br>mug<br>must<br>my<br>myself<br>n<br>na<br>name<br>namely<br>nay<br>nd<br>near<br>nearly<br>necessarily<br>necessary<br>need<br>needs<br>neither<br>never<br>nevertheless<br>new<br>next<br>nine<br>ninety<br>no<br>nobody<br>non<br>none<br>nonetheless<br>noone<br>nor<br>normally<br>nos<br>not<br>noted<br>nothing<br>now<br>nowhere<br>o<br>obtain<br>obtained<br>n't<br>doesn<br>obviously<br>of<br>off<br>often<br>oh<br>ok<br>okay<br>old<br>omitted<br>on<br>once<br>one<br>ones<br>only<br>onto<br>or<br>ord<br>other<br>others<br>otherwise<br>ought<br>our<br>ours<br>ourselves<br>out<br>outside<br>over<br>overall<br>owing<br>own<br>p<br>page<br>pages<br>part<br>particular<br>particularly<br>past<br>per<br>perhaps<br>placed<br>please<br>plus<br>poorly<br>possible<br>possibly<br>potentially<br>pp<br>predominantly<br>present<br>previously<br>primarily<br>probably<br>promptly<br>proud<br>provides<br>put<br>q<br>que<br>quickly<br>quite<br>qv<br>r<br>ran<br>rather<br>rd<br>re<br>readily<br>really<br>recent<br>recently<br>ref<br>refs<br>regarding<br>regardless<br>regards<br>related<br>relatively<br>research<br>respectively<br>resulted<br>resulting<br>results<br>right<br>run<br>s<br>said<br>same<br>saw<br>say<br>saying<br>says<br>sec<br>section<br>see<br>seeing<br>seem<br>seemed<br>seeming<br>seems<br>seen<br>self<br>selves<br>sent<br>seven<br>several<br>shall<br>she<br>shed<br>she'll<br>shes<br>should<br>shouldn't<br>show<br>showed<br>shown<br>showns<br>shows<br>significant<br>significantly<br>similar<br>similarly<br>since<br>six<br>slightly<br>so<br>some<br>somebody<br>somehow<br>someone<br>somethan<br>something<br>sometime<br>sometimes<br>somewhat<br>somewhere<br>soon<br>sorry<br>specifically<br>specified<br>specify<br>specifying<br>still<br>stop<br>strongly<br>sub<br>substantially<br>successfully<br>such<br>sufficiently<br>suggest<br>sup<br>sure<br>t<br>take<br>taken<br>taking<br>tell<br>tends<br>th<br>than<br>thank<br>thanks<br>thanx<br>that<br>that'll<br>thats<br>that've<br>the<br>their<br>theirs<br>them<br>themselves<br>then<br>thence<br>there<br>thereafter<br>thereby<br>thered<br>therefore<br>therein<br>there'll<br>thereof<br>therere<br>theres<br>thereto<br>thereupon<br>there've<br>these<br>they<br>theyd<br>they'll<br>theyre<br>they've<br>think<br>this<br>those<br>thou<br>though<br>thoughh<br>thousand<br>throug<br>through<br>throughout<br>thru<br>thus<br>til<br>tip<br>to<br>together<br>too<br>took<br>toward<br>towards<br>tried<br>tries<br>truly<br>try<br>trying<br>ts<br>twice<br>two<br>u<br>un<br>under<br>unfortunately<br>unless<br>unlike<br>unlikely<br>until<br>unto<br>up<br>upon<br>ups<br>us<br>use<br>used<br>useful<br>usefully<br>usefulness<br>uses<br>using<br>usually<br>v<br>value<br>various<br>'ve<br>very<br>via<br>viz<br>vol<br>vols<br>vs<br>w<br>want<br>wants<br>was<br>wasnt<br>way<br>we<br>wed<br>welcome<br>we'll<br>went<br>were<br>werent<br>we've<br>what<br>whatever<br>what'll<br>whats<br>when<br>whence<br>whenever<br>where<br>whereafter<br>whereas<br>whereby<br>wherein<br>wheres<br>whereupon<br>wherever<br>whether<br>which<br>while<br>whim<br>whither<br>who<br>whod<br>whoever<br>whole<br>who'll<br>whom<br>whomever<br>whos<br>whose<br>why<br>widely<br>willing<br>wish<br>with<br>within<br>without<br>wont<br>words<br>world<br>would<br>wouldnt<br>www<br>x<br>y<br>yes<br>yet<br>you<br>youd<br>you'll<br>your<br>youre<br>yours<br>yourself<br>yourselves<br>you've<br>z<br>zero"
stopwords = longStopwordList.split("<br>")

stopwords = list(map(converter, stopwords))

In [3]:
#Read dataset line by line for values in “review/text” field.
topWords = []
topWordsCount = []
reviews = []
reviewsTokenList = []

with open("foods.txt") as fp:
    for line in fp:
        
        if(line.startswith("review/text: ")):
            each_line = line.split("review/text: ")[1]
            reviews.append(each_line)
            reviewsTokenList.extend(list(map(converter, wordpunct_tokenize(each_line))))

L = list(set([word for word in reviewsTokenList if word !="" and not word.isnumeric() and word!= "br"]))

#Clean set L removing stopwards and saving to W
W = [word for word in L if word not in stopwords]

#Count the number of times each word in W appears among all reviews
freqDistribution = FreqDist(reviewsTokenList)

#Identify the top 500 words 
count = 0 
for word, value in freqDistribution.most_common():
    if(word in W):
        topWords.append(word)
        topWordsCount.append(value)
        count = count + 1
        if(count == 500):
            break

In [4]:
#Vectorize all reviews (“review/text” field) using these 500 words
vectorizer = CountVectorizer(vocabulary = topWords)
V = vectorizer.fit_transform(reviews)

In [5]:
#Cluster the vectorized reviews into 10 clusters using k-means.
km = KMeans(n_clusters=10, random_state = 0).fit(normalize(V))

In [6]:
#From each centroid, select the top 5 words that represent the centroid
outputWords = []
featureValues = []

sortCenter = km.cluster_centers_.argsort()[:,::-1]

for i in range(len(sortCenter)):
    
    wordlist = []
    featureList = []
    
    for word in list(sortCenter[i, :5]):
        
        wordlist.append(topWords[word])
        featureList.append(km.cluster_centers_[i,word])
        
    outputWords.append(wordlist)
    featureValues.append(featureList)
    
# Print Output Results

print("\nTop 500 Words and counts of these words:\n")
for i in range(0,500):
    print(str(topWords[i]) + str("  -  ") + str(topWordsCount[i]))
    
print("\nTop 5 words representing each cluster and  their feature values:")
for i in range(0,10):
    print("\nCluster "+str(i+1) + " :\n")
    print(outputWords[i]) 
    print(featureValues[i])


Top 500 Words and counts of these words:

taste  -  218479
good  -  202411
product  -  200183
flavor  -  187628
coffee  -  176327
great  -  167190
love  -  164014
tea  -  153863
food  -  153386
will  -  127176
dog  -  107705
amazon  -  106609
time  -  101643
don  -  91874
cup  -  90354
bag  -  87983
price  -  81127
buy  -  77371
best  -  76862
find  -  74145
well  -  73467
better  -  70772
day  -  68477
eat  -  67787
treat  -  67746
store  -  67568
box  -  66548
cat  -  64918
chocolate  -  64174
water  -  62752
drink  -  61147
year  -  60619
sugar  -  60336
brand  -  58981
sweet  -  54119
free  -  50727
bought  -  50341
thing  -  49759
bit  -  47236
order  -  46380
favorite  -  45246
ingredient  -  44937
mix  -  44377
chip  -  44090
lot  -  43908
snack  -  43322
recommend  -  42674
bar  -  42490
work  -  42404
pack  -  41788
nice  -  41520
delicious  -  41476
add  -  39691
hot  -  38355
stuff  -  37327
review  -  36902
organic  -  36606
quality  -  36148
healthy  -  35722
didn  -  353